In [1]:
import os
import time
import dask_cudf

from dask.distributed import Client, wait, get_worker
from dask_snowflake import read_snowflake
from dask_cuda import LocalCUDACluster

In [2]:
cluster = LocalCUDACluster(threads_per_worker=8, n_workers=1)
client = Client(cluster)
client.scheduler_info()

/opt/conda/envs/rapids/lib/python3.8/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33969 instead
  warnings.warn(
2022-04-28 17:36:38,612 - distributed.diskutils - INFO - Found stale lock file and directory '/rapids/notebooks/snowflake/dask-worker-space/worker-tj4ysty3', purging
2022-04-28 17:36:38,613 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


{'type': 'Scheduler',
 'id': 'Scheduler-14b781e4-d393-42ad-b09d-29230d0877eb',
 'address': 'tcp://127.0.0.1:34653',
 'services': {'dashboard': 33969},
 'started': 1651167397.2721257,
 'workers': {'tcp://127.0.0.1:35061': {'type': 'Worker',
   'id': 0,
   'host': '127.0.0.1',
   'resources': {},
   'local_directory': '/rapids/notebooks/snowflake/dask-worker-space/worker-wkeovr2d',
   'name': 0,
   'nthreads': 8,
   'memory_limit': 540656029696,
   'last_seen': 1651167398.8078148,
   'services': {'dashboard': 35023},
   'metrics': {'executing': 0,
    'in_memory': 0,
    'ready': 0,
    'in_flight': 0,
    'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}},
    'spilled_nbytes': {'memory': 0, 'disk': 0},
    'cpu': 0.0,
    'memory': 234074112,
    'time': 1651167398.621196,
    'read_bytes': 0.0,
    'write_bytes': 0.0,
    'read_bytes_disk': 0.0,
    'write_bytes_disk': 0.0,
    'num_fds': 26,
    'gpu_utilization': 0,
    'gpu_memory_used': 8725659648,
    'gpu': {'utilization': 2, 'memory-used': 8995143680}},
   'nanny': 'tcp://127.0.0.1:40161',
   'gpu': {'memory-total': 51527024640, 'name': 'NVIDIA RTX A6000'}}}}

In [3]:
from dotenv import load_dotenv

config = load_dotenv()

DB_CREDS = {
    "user": os.environ["SNOW_USER"],
    "password": os.environ["SNOW_PASSWORD"],
    "account": os.environ["SNOW_ACCOUNT"],
    "database": "SAMPLE_DATA",
    "warehouse": os.environ["SNOW_WAREHOUSE"]
}

QUERY = """
SELECT 
    ss.SS_ITEM_SK, ss.SS_STORE_SK, s.S_STATE,
    d.D_DATE, ss.SS_QUANTITY, ss.SS_EXT_WHOLESALE_COST, 
    ss.SS_EXT_LIST_PRICE, ss.SS_NET_PROFIT  
FROM 
    SAMPLE_DATA.TPCDS_SF100TCL.STORE_SALES ss, 
    SAMPLE_DATA.TPCDS_SF100TCL.STORE s,
    SAMPLE_DATA.TPCDS_SF100TCL.DATE_DIM d
WHERE 
    ss.SS_STORE_SK = s.S_STORE_SK and
    ss.SS_SOLD_DATE_SK = d.D_DATE_SK
LIMIT 10000000
"""

In [4]:
t1 = time.time()
ddf = read_snowflake(
    query=QUERY,
    connection_kwargs=DB_CREDS
)
t2 = time.time()
t2 - t1

2.1030046939849854

In [5]:
t1 = time.time()
len(ddf)
t2 = time.time()
t2 - t1

16.334730625152588

In [6]:
t1 = time.time()
ddf = dask_cudf.from_dask_dataframe(ddf)
t2 = time.time()
t2 - t1

0.599092960357666

In [7]:
t1 = time.time()
len(ddf)
t2 = time.time()
t2 - t1

23.459553480148315